In [ ]:
import glob, os, re

In [ ]:
# path to working directory
root = os.getcwd()
print root# print(root) for Py3 because print has become a function

# path to data directory
dpath = os.path.join(root,"DATA","INAUG")

print os.listdir(dpath)

In [ ]:
def read_txt(path):
    fnames = sorted(glob.glob(os.path.join(path,"*.txt")))
    data = []
    metadata = []
    for fname in fnames:
        with open(fname, 'r') as f:
            data.append(f.read())
        metadata.append(os.path.basename(fname).split(".")[0])
    return data, metadata

def clean(s):
    return re.sub(r"\d","",s.lower())

In [ ]:
texts, titles = read_txt(dpath)
print texts[-1]

## and scikit-learn happens ##
import vectorizer and model object

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

Transform texts to vector space

In [ ]:
vectorizer = CountVectorizer(max_df = 0.95, min_df = 0.01, 
                             max_features = 500, stop_words = 'english',
                            encoding = 'ascii', ngram_range = (1,1))

X = vectorizer.fit_transform(texts)
vocab = vectorizer.get_feature_names()
X.shape

## train model ##

In [26]:
k = 5
mdl1 = LatentDirichletAllocation(n_topics = k, max_iter = 100, 
                                 random_state = 1234, 
                                 learning_method = 'online').fit(X)

### exploring the model ###

In [24]:
def display_topics(model, feature_names, no_top_words = 10):
    for topic_idx, topic in enumerate(model.components_):
        print "Topic {}".format(topic_idx)
        print " ".join([feature_names[i] for i in topic.argsort()
                        [:-no_top_words-1:-1]])
        
vocab = vectorizer.get_feature_names()
print len(vocab)

display_topics(mdl1, vocab)

500
Topic 0
world new america nation government let peace time today great
Topic 1
government states constitution union great shall power citizens united congress
Topic 2
government country public shall great nations states peace citizens national
Topic 3
government country states great public constitution shall peace free nations
Topic 4
freedom world free nation america nations know peace life men
